In [ ]:
import asyncio
import logging
import sys

from astropy.coordinates import Angle
from astropy.time import Time
import astropy.units as u

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS
from lsst.ts.observatory.control.maintel.comcam import ComCam

import mt_utils

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
#mtcs = MTCS(domain)
comcam = ComCam(domain)
mtptg = salobj.Remote(domain, "MTPtg")
newmtmount = salobj.Remote(domain, "NewMTMount")

#await asyncio.gather(mtcs.start_task, comcam.start_task)
await asyncio.gather(comcam.start_task, mtptg.start_task, newmtmount.start_task)

In [ ]:
ra_dec = mt_utils.altaz_to_radec(80.0, 0.0)

mtptg.cmd_raDecTarget.set(
    ra=ra_dec.ra.hour,
    declination=ra_dec.dec.deg,
    rotPA=Angle(0.0, unit=u.deg).deg,
    targetName="Park position",
    frame=2,
    epoch=2000,
    equinox=2000,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotFrame=1,
    rotMode=1,
)

# mtptg.cmd_azElTarget.set(
#     targetName="Park position",
#     azDegs=Angle(0.0, unit=u.deg).deg,
#     elDegs=Angle(80.0, unit=u.deg).deg,
#     rotPA=Angle(0.0, unit=u.deg).deg
# )

In [ ]:
await mt_utils.slew(mtptg, newmtmount, True)
# ack = await mtptg.cmd_azElTarget.start(timeout=30)
#print(f"ack={ack.ack} ack.error={ack.error}, ackcmd.result={ack.result}")

In [ ]:
await comcam.standby()